In [1]:
import sys
sys.path.insert(0, "..")

from pathlib import Path

from src.data import make_dataset
from src.models import train_model
from src.models import metrics

import torch
from torch import nn
import torch.nn as nn
import torch.optim as optim

import gin
sys.path.insert(0, "../..")

2022-12-11 15:44:30.873410: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-11 15:44:31.001620: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-11 15:44:31.001650: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-11 15:44:31.747820: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
datadir = Path("../../data/raw/")
train_dataloader, test_dataloader = make_dataset.get_MNIST(datadir, batch_size=64) 
len(train_dataloader), len(test_dataloader)

(938, 157)

In [ ]:
gin.parse_config_file("model.gin")

In [3]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.convolutions = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        
        self.dense = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        x = self.convolutions(x)
        logits = self.dense(x)
        return logits

model = CNN().to(device)
#print(model)

Using cpu device


In [4]:
x, y = next(iter(train_dataloader))
x.shape, y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [5]:
train_model.count_parameters(model)

optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [ ]:
import torch.optim as optim
gin.parse_config_file("model.gin")

units = [512, 256, 128]
learning_rate = [0.01, 0.001, 0.0001]

for unit in units:
    for lr in learning_rate:
        gin.bind_parameter("NeuralNetwork.units2", unit)
        gin.bind_parameter("trainloop.learning_rate", lr)
    
        model = imagemodels.NeuralNetwork()

        model =  train_model.trainloop(
            epochs=10,
            model=model,
            metrics=[accuracy],
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            train_steps=len(train_dataloader),
            eval_steps=150,
        )

In [7]:
model = train_model.trainloop(
    epochs=3,
    model=model,
    optimizer=optimizer,
    learning_rate=1e-3,
    loss_fn=loss_fn,
    metrics=[accuracy],
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    log_dir="../../models/test/",
    train_steps=len(train_dataloader),
    eval_steps=len(test_dataloader),
)

2022-12-11 15:13:43.346 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/test/20221211-1513
100%|██████████| 938/938 [00:28<00:00, 32.78it/s]
2022-12-11 15:14:14.725 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.5274 test 0.4557 metric ['0.8413']
100%|██████████| 938/938 [00:28<00:00, 33.10it/s]
2022-12-11 15:14:45.654 | INFO     | src.models.train_model:trainloop:171 - Epoch 1 train 0.4023 test 0.3920 metric ['0.8610']
100%|██████████| 938/938 [00:28<00:00, 32.82it/s]
2022-12-11 15:15:16.775 | INFO     | src.models.train_model:trainloop:171 - Epoch 2 train 0.3486 test 0.3621 metric ['0.8670']
100%|██████████| 3/3 [01:33<00:00, 31.09s/it]
